In [1]:
import json
import pandas
from sqlalchemy import create_engine

In [19]:
import os
from dotenv import load_dotenv

load_dotenv(verbose=True)
REDSHIFT_ENGINE = os.environ.get('REDSHIFT_ENGINE')

In [20]:
test_query = """
    select
        date_trunc('month', cast(metrc_sales_receipts.sales_datetime as timestamp)) as date_type,
        metrc_sales_transactions.product_category_name,
        sum(metrc_sales_transactions.total_price)
    from
        companies
        inner join metrc_sales_receipts on metrc_sales_receipts.company_id = metrc_sales_receipts.company_id
        inner join metrc_sales_transactions on metrc_sales_receipts.id = metrc_sales_transactions.receipt_row_id
    where
        True
        and companies.identifier = 'RA'
        and metrc_sales_receipts.sales_datetime >= '2020-01-01'
    group by
        date_trunc('month', cast(metrc_sales_receipts.sales_datetime as timestamp)),
        metrc_sales_transactions.product_category_name
    order by
        date_type asc,
        product_category_name asc
"""

incoming_transfer_packages_query = """
    select
        metrc_transfers.id as transfer_row_id,
        metrc_deliveries.id as delivery_row_id,
        metrc_transfer_packages.id as package_row_id,
        case
            when company_deliveries.delivery_type = 'INCOMING_UNKNOWN' then 'INCOMING_FROM_VENDOR'
            when company_deliveries.delivery_type = 'INCOMING_FROM_VENDOR' then 'INCOMING_FROM_VENDOR'
            when company_deliveries.delivery_type = 'OUTGOING_UNKNOWN' then 'OUTGOING_TO_PAYOR'
            when company_deliveries.delivery_type = 'OUTGOING_TO_PAYOR' then 'OUTGOING_TO_PAYOR'
            else company_deliveries.delivery_type
        end as delivery_type,
        company_deliveries.license_number,
        metrc_transfers.manifest_number,
        metrc_transfers.created_date,
        metrc_deliveries.received_datetime,
        metrc_transfers.shipper_facility_license_number,
        metrc_transfers.shipper_facility_name,
        metrc_deliveries.recipient_facility_license_number,
        metrc_deliveries.recipient_facility_name,
        metrc_deliveries.shipment_type_name,
        metrc_deliveries.shipment_transaction_type,
        metrc_transfer_packages.package_id,
        metrc_transfer_packages.package_label,
        metrc_transfer_packages.type,
        metrc_transfer_packages.package_payload_shipmentpackagestate as shipment_package_state,
        metrc_transfer_packages.package_payload_istestingsample as is_testing_sample,
        metrc_transfer_packages.package_payload_istradesample as is_trade_sample,
        metrc_transfer_packages.product_category_name,
        metrc_transfer_packages.product_name,
        metrc_transfer_packages.lab_results_status as package_lab_results_status,
        metrc_transfer_packages.shipper_wholesale_price,
        metrc_transfer_packages.shipped_quantity,
        metrc_transfer_packages.shipped_unit_of_measure,
        metrc_transfer_packages.received_quantity,
        metrc_transfer_packages.received_unit_of_measure,
        metrc_transfer_packages.package_payload_itemunitweight as item_unit_weight,
        metrc_transfer_packages.package_payload_itemunitweightunitofmeasurename as item_unit_weight_unit_of_measure_name
    from
        metrc_transfers
        inner join company_deliveries on metrc_transfers.id = company_deliveries.transfer_row_id
        inner join companies on company_deliveries.company_id = companies.id
        inner join metrc_deliveries on metrc_transfers.id = metrc_deliveries.transfer_row_id
        inner join metrc_transfer_packages on metrc_deliveries.id = metrc_transfer_packages.delivery_row_id
    where
        True
        and companies.identifier = 'HPCC'
        and (
            company_deliveries.delivery_type = 'INCOMING_FROM_VENDOR' or
            company_deliveries.delivery_type = 'INCOMING_UNKNOWN'
        )
        and metrc_transfers.created_date >= '2020-01-01'
    order by
        created_date desc
"""

sales_transactions_query = """
    select
        metrc_sales_receipts.id,
        metrc_sales_transactions.last_modified_at,
        metrc_sales_receipts.license_number,
        metrc_sales_receipts.receipt_id,
        metrc_sales_receipts.receipt_number,
        metrc_sales_receipts.type as receipt_type,
        metrc_sales_receipts.sales_customer_type,
        metrc_sales_receipts.sales_datetime,
        metrc_sales_receipts.total_packages,
        metrc_sales_receipts.total_price,
        metrc_sales_transactions.type as tx_type,
        metrc_sales_transactions.package_id as tx_package_id,
        metrc_sales_transactions.package_label as tx_package_label,
        metrc_sales_transactions.product_name as tx_product_name,
        metrc_sales_transactions.product_category_name as tx_product_category_name,
        metrc_sales_transactions.unit_of_measure as tx_unit_of_measure,
        metrc_sales_transactions.quantity_sold as tx_quantity_sold,
        metrc_sales_transactions.total_price as tx_total_price
    from
        metrc_sales_receipts
        inner join companies on metrc_sales_receipts.company_id = companies.id
        inner join metrc_sales_transactions on metrc_sales_receipts.id = metrc_sales_transactions.receipt_row_id
    where
        True
        and companies.identifier = 'HPCC'
        and metrc_sales_receipts.sales_datetime >= '2021-01-01'
    order by
        metrc_sales_receipts.sales_datetime desc
"""

redshift_engine = create_engine(REDSHIFT_ENGINE)
test = pandas.read_sql_query(sales_transactions_query, redshift_engine)

In [21]:
test

,id,last_modified_at,license_number,receipt_id,receipt_number,receipt_type,sales_customer_type,sales_datetime,total_packages,total_price,tx_type,tx_package_id,tx_package_label,tx_product_name,tx_product_category_name,tx_unit_of_measure,tx_quantity_sold,tx_total_price
0,7b186405-d716-4af2-9e60-fae1bb295c46,2021-10-07 16:54:21,C10-0000064-LIC,134527995,0134527995,active,Consumer,2021-10-07 09:54:20,3,66.44,active,18256600,1A4060300007DCA000024336,Ice Cream Cake 1/4 oz,Flower (packaged quarter - each),Each,1.0,21.84
1,7b186405-d716-4af2-9e60-fae1bb295c46,2021-10-07 16:54:21,C10-0000064-LIC,134527995,0134527995,active,Consumer,2021-10-07 09:54:20,3,66.44,active,18350637,1A406030000E80A000448921,17185 Union Electric Flower 3.5g Pouch Indica ...,Flower (packaged eighth - each),Each,1.0,26.21
2,7b186405-d716-4af2-9e60-fae1bb295c46,2021-10-07 16:54:21,C10-0000064-LIC,134527995,0134527995,active,Consumer,2021-10-07 09:54:20,3,66.44,active,18157551,1A4060300003D59000020382,Cali Flwr Farm - SOFA King PURPS (3.5 GM Flowe...,Flower (packaged eighth - each),Each,1.0,18.39
3,09feda84-0420-47b2-a088-f61e7d60e10e,2021-10-07 16:52:08,C10-0000064-LIC,134527632,0134527632,active,Consumer,2021-10-07 09:52:07,2,180.72,active,17552512,1A406030001989C000005240,Jelly Wizard Edibles Rosin Jellies,Edible (weight - each),Each,3.0,55.68
4,09feda84-0420-47b2-a088-f61e7d60e10e,2021-10-07 16:52:08,C10-0000064-LIC,134527632,0134527632,active,Consumer,2021-10-07 09:52:07,2,180.72,active,18355213,1A4060300003D59000020655,Cali Flwr Farms - Cali Gas OG (3.5 GM Flower (...,Flower (packaged eighth - each),Each,8.0,125.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274334,f33d1327-13cc-4ac9-a1ef-9b0de829bc3e,2021-01-01 20:08:27,C10-0000064-LIC,None,0070529106,inactive,Consumer,2021-01-01 12:02:43,2,49.43,inactive,10403405,1A406030000E80A000194522,11369 Pacific Stone Flower 3.5g Pouch Hybrid 8...,Flower,Grams,3.5,21.84
274335,f33d1327-13cc-4ac9-a1ef-9b0de829bc3e,2021-01-01 20:08:27,C10-0000064-LIC,None,0070529106,inactive,Consumer,2021-01-01 12:02:43,2,49.43,inactive,10316753,1A40603000070EA000255392,LIIIL - OG Kush Disposable - 0.5g,Vape Cartridge (volume - each),Each,1.0,27.59
274336,8e40c318-e4dd-48af-8005-e2b4f075eaec,2021-01-01 20:08:23,C10-0000064-LIC,None,0070529099,inactive,Consumer,2021-01-01 12:01:17,1,76.96,inactive,10292886,1A406030000ECBD000001203,S.F. Roots - Chocolatina - Flower - 3.5g - Cas...,Flower (packaged eighth - each),Each,2.0,76.96
274337,1f5eec8a-84d9-4fea-a01a-ba0cdefa4555,2021-01-01 20:04:18,C10-0000064-LIC,None,0070528378,inactive,Consumer,2021-01-01 11:56:00,2,108.00,inactive,9858991,1A4060300006852000003804,Eighth - ZIABLO - 3.5 g,Flower,Grams,3.5,54.00
